## python 对C语言的扩展  
  
**使用 C 或 C++ 扩展 Python 的知识是读懂pytorch源码的关键。因为pytorch的很多类都是继承自c++语言编码的模块**    
  
#### 在python文件中调用C语言编写的文件  
 如：spam模块是c++语言编写的模块。现在想在python文件中引入这个模块，并直接使用这个模块里的函数。  

```python
 import spam  
 status = spam.system("ls -l")  
``` 
 
<br>  
 
我们的目的是想要达到上面要求。  
  
<br>  
  
```python
 import spam 
```
这个模块是C语言模块。要在python中能使用这个模块，则必定要使用Python的扩展规范来导出这个符号。（如果是Python2，会使用Py_InitModule API；如果是Python3的话，则会使用PyModule_Create API；）   
  
<br>   
  
<br>  
   
##  使用Python的扩展规范，让python文件可以调用c模块    
  
  
 ```python
 import spam  
 status = spam.system("ls -l")  
``` 
 
 这里spam是一个c语言写的模块system是模块里的函数。  
   
<br>  

1、创建模块文件  $\quad$ spammodule.c  
  
```C 
# 第一行获取python 的API 
# include <python.h>   
  
#创建模块结构体 （用PyModuleDef结构体 声明了 torchmodule 变量模块） 
  static struct PyModuleDef torchmodule = {
     PyModuleDef_HEAD_INIT,
     "spam",             ## 模块名称
     nullptr,
     -1,                ##模块保持全局初始化
     SpanMethods        ##模块里包含的变量和方法的数组（类型是PyMethodDef结构体）
  };
   
  
  
#创建方法结构体数组  （用PyMethodDef结构体 声明一个 PyMethodDef 的数组 ） 
static PyMethodDef SpanMethods[]={
    ……  
        
    # 这里第一system 是函数spam_system在模块中的名字。METH_VARARGS是在c和python之间传递参数的
    {"system",spam_system,METH_VARARGS,"Execute a shell command."}    
        
    ……
    
}  
  
# 创建模块里的方法  
static PyObject * spam_system(PyObject *self,PyObject * args)
{
   ……
}


#整个模块都定义好了，结构体有了，方法有了。可以开始定义整个模块的初始化函数了。 
PyMODINIT_FUNC PyInit_spam(void)
{
    return PyModule_Create(&spammodule)
   
}


```

以上完成了一个c模块的定义。编译之后。就可以在python中调用了。  
  
  

  